## Part 1

First, we are going to import requests library. Then, we will store the data into Pandas Dataframe.

Import Libraries.

In [112]:
import requests
import pandas as pd
import numpy as np

Assign a link of the website through which we are going to scrape the data to a variable named 'wikitable'.

In [94]:
wikitable = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [101]:
# Storing data into panda dataframe
df_can=pd.read_html(wikitable,keep_default_na=False)[0]

In [98]:
df_can.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [99]:
df_can.shape

(288, 3)

Drop cells with a borough that is 'Not assigned'

In [128]:
df_canada = df_can[~df_can['Borough'].isin(['Not assigned'])]
df_canada.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [119]:
df_canada.shape

(211, 3)

If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough. 

In [137]:
df_canada.Neighbourhood[df_canada.Neighbourhood=='Not assigned']=df_canada.Borough
df_canada.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


More than one neighborhood can exist in one postal code area. For example, M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [135]:
canada_df = df_canada.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]).reset_index()

In [157]:
# check the result
canada_df.loc[canada_df['Postcode'] == 'M5A']

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [187]:
# rename a column to match Coursera naming
canada_df.rename(columns={'Postcode': 'Postal Code'}).head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [153]:
print('Number of rows in the dataframe',canada_df.shape)

Number of rows in the dataframe (103, 3)


## Part 2

Get geographical coordinates for each postal code, then merge all data into one dataframe.

In [208]:
#Read csv file that has the geographical coordinates of each postal code: 'http://cocl.us/Geospatial_data'

geo_data=pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [209]:
#concatinate two dataframes
con_df = pd.concat([canada_df,geo_data], axis=1)
con_df.head()

# drop extra "Postal Code" column
canada_data=con_df.drop(['Postal Code'] , axis=1)
canada_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [210]:
canada_data.shape

(103, 5)